In [1]:
import os
import copy
import time
import torch
import warnings
import numpy as np
import torch.nn as nn
from src.utils import *
from src.loss import YoloLoss
from src.yolo_net import Yolo
from src.voc_dataset import VOCDataset
from torch.utils.data import DataLoader

In [2]:
warnings.simplefilter("ignore")

In [3]:
num_classes = 20
if torch.cuda.is_available():
    torch.cuda.manual_seed(369)
else:
    torch.manual_seed(369)

#learning_rate_schedule = {"0": 1e-5, "5": 1e-4,
#                          "80": 1e-5, "110": 1e-6}
learning_rate_schedule = {"0": 1e-5, "5": 1e-4,
                          "25": 1e-5, "50": 1e-6}
training_params = {"batch_size": 32,
                    "shuffle": True,
                    "drop_last": True,
                    "collate_fn": custom_collate_fn}

test_params = {"batch_size": 32,
                "shuffle": False,
                "drop_last": False,
                "collate_fn": custom_collate_fn}

training_set = VOCDataset('/mnt/YOLOv1/dataset/VOC2007train', image_size=448, is_training = True)
training_generator = DataLoader(training_set, **training_params)

test_set = VOCDataset('/mnt/YOLOv1/dataset/VOC2007test', image_size=448, is_training=False)
test_generator = DataLoader(test_set, **test_params)

In [4]:
model = Yolo(num_classes)
model_dict = model.state_dict()
pretrained_dict = torch.load('trained_models/yolov2_coco_pretrained.pth')
pretrained_dict = {k: v for k, v in pretrained_dict.items() if np.shape(model_dict[k]) ==  np.shape(v)}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
# The following line will re-initialize weight for the last layer
nn.init.normal_(list(model.modules())[-1].weight, 0, 0.01)
if torch.cuda.is_available():
    model.cuda()
criterion = YoloLoss(num_classes, model.anchors)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9, weight_decay=0.0005)

In [5]:
best_loss = 1e10
best_model_wts = copy.deepcopy(model.state_dict())
num_epoches = 25
num_iter_per_epoch = len(training_generator)
for epoch in range(num_epoches):
    model.train()
    if str(epoch) in learning_rate_schedule.keys():
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate_schedule[str(epoch)]
    for iter, batch in enumerate(training_generator):
        image, label = batch
        if torch.cuda.is_available():
            image = Variable(image.cuda(), requires_grad=True)
        else:
            image = Variable(image, requires_grad=True)
        optimizer.zero_grad()
        logits = model(image)
        loss, loss_coord, loss_conf, loss_cls = criterion(logits, label)
        loss.backward()
        optimizer.step()
        if iter % 10 == 0:
            with open('train_loss.csv', 'a+') as f:
                f.write('{0:.2f}'.format(loss) + '\n')
            print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Train Loss:{:.2f} (Coord:{:.2f} Conf:{:.2f} Cls:{:.2f})".format(
                epoch + 1,
                num_epoches,
                iter + 1,
                num_iter_per_epoch,
                optimizer.param_groups[0]['lr'],
                loss,
                loss_coord,
                loss_conf,
                loss_cls))
        if loss < best_loss:
            best_loss = loss
            best_model_wts = copy.deepcopy(model.state_dict())
'''
    if epoch % 10 == 0:
        model.eval()
        loss_ls = []
        loss_coord_ls = []
        loss_conf_ls = []
        loss_cls_ls = []
        for te_iter, te_batch in enumerate(test_generator):
            te_image, te_label = te_batch
            num_sample = len(te_label)
            if torch.cuda.is_available():
                te_image = te_image.cuda()
            with torch.no_grad():
                te_logits = model(te_image)
                batch_loss, batch_loss_coord, batch_loss_conf, batch_loss_cls = criterion(te_logits, te_label)
            loss_ls.append(batch_loss * num_sample)
            loss_coord_ls.append(batch_loss_coord * num_sample)
            loss_conf_ls.append(batch_loss_conf * num_sample)
            loss_cls_ls.append(batch_loss_cls * num_sample)
        te_loss = sum(loss_ls) / test_set.__len__()
        te_coord_loss = sum(loss_coord_ls) / test_set.__len__()
        te_conf_loss = sum(loss_conf_ls) / test_set.__len__()
        te_cls_loss = sum(loss_cls_ls) / test_set.__len__()
        print("Epoch: {}/{}, Lr: {}, Test Loss:{:.2f} (Coord:{:.2f} Conf:{:.2f} Cls:{:.2f})".format(
            epoch + 1,
            num_epoches,
            optimizer.param_groups[0]['lr'],
            te_loss,
            te_coord_loss,
            te_conf_loss,
            te_cls_loss))
        if te_loss < best_loss:
            best_loss = te_loss
            best_model_wts = copy.deepcopy(model.state_dict())
'''
torch.save(best_model_wts, 'trained_models/yolov2_voc_weights0.pth')

Epoch: 1/25, Iteration: 1/156, Lr: 1e-05, Train Loss:241.66 (Coord:1.75 Conf:222.38 Cls:17.54)
Epoch: 1/25, Iteration: 11/156, Lr: 1e-05, Train Loss:43.35 (Coord:1.81 Conf:22.96 Cls:18.58)
Epoch: 1/25, Iteration: 21/156, Lr: 1e-05, Train Loss:25.57 (Coord:1.61 Conf:6.25 Cls:17.71)
Epoch: 1/25, Iteration: 31/156, Lr: 1e-05, Train Loss:17.47 (Coord:1.05 Conf:3.92 Cls:12.49)
Epoch: 1/25, Iteration: 41/156, Lr: 1e-05, Train Loss:20.01 (Coord:1.38 Conf:4.22 Cls:14.41)
Epoch: 1/25, Iteration: 51/156, Lr: 1e-05, Train Loss:19.60 (Coord:1.12 Conf:4.16 Cls:14.32)
Epoch: 1/25, Iteration: 61/156, Lr: 1e-05, Train Loss:23.05 (Coord:1.55 Conf:4.57 Cls:16.94)
Epoch: 1/25, Iteration: 71/156, Lr: 1e-05, Train Loss:19.33 (Coord:1.02 Conf:4.26 Cls:14.05)
Epoch: 1/25, Iteration: 81/156, Lr: 1e-05, Train Loss:24.94 (Coord:1.61 Conf:5.10 Cls:18.22)
Epoch: 1/25, Iteration: 91/156, Lr: 1e-05, Train Loss:22.70 (Coord:1.28 Conf:5.32 Cls:16.09)
Epoch: 1/25, Iteration: 101/156, Lr: 1e-05, Train Loss:23.84 (Coord

In [4]:
model = Yolo(num_classes)
model.cuda()
model.load_state_dict(torch.load('trained_models/yolov2_voc_weights0.pth'))
criterion = YoloLoss(num_classes, model.anchors)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9, weight_decay=0.0005)

best_loss = 1e10
best_model_wts = copy.deepcopy(model.state_dict())
num_epoches = 50
num_iter_per_epoch = len(training_generator)
for epoch in range(25, num_epoches):
    model.train()
    if str(epoch) in learning_rate_schedule.keys():
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate_schedule[str(epoch)]
    for iter, batch in enumerate(training_generator):
        image, label = batch
        if torch.cuda.is_available():
            image = Variable(image.cuda(), requires_grad=True)
        else:
            image = Variable(image, requires_grad=True)
        optimizer.zero_grad()
        logits = model(image)
        loss, loss_coord, loss_conf, loss_cls = criterion(logits, label)
        loss.backward()
        optimizer.step()
        if iter % 10 == 0:
            with open('train_loss.csv', 'a+') as f:
                f.write('{0:.2f}'.format(loss) + '\n')
            print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Train Loss:{:.2f} (Coord:{:.2f} Conf:{:.2f} Cls:{:.2f})".format(
                epoch + 1,
                num_epoches,
                iter + 1,
                num_iter_per_epoch,
                optimizer.param_groups[0]['lr'],
                loss,
                loss_coord,
                loss_conf,
                loss_cls))
        if loss < best_loss:
            best_loss = loss
            best_model_wts = copy.deepcopy(model.state_dict())
'''
    if epoch % 10 == 0:
        model.eval()
        loss_ls = []
        loss_coord_ls = []
        loss_conf_ls = []
        loss_cls_ls = []
        for te_iter, te_batch in enumerate(test_generator):
            te_image, te_label = te_batch
            num_sample = len(te_label)
            if torch.cuda.is_available():
                te_image = te_image.cuda()
            with torch.no_grad():
                te_logits = model(te_image)
                batch_loss, batch_loss_coord, batch_loss_conf, batch_loss_cls = criterion(te_logits, te_label)
            loss_ls.append(batch_loss * num_sample)
            loss_coord_ls.append(batch_loss_coord * num_sample)
            loss_conf_ls.append(batch_loss_conf * num_sample)
            loss_cls_ls.append(batch_loss_cls * num_sample)
        te_loss = sum(loss_ls) / test_set.__len__()
        te_coord_loss = sum(loss_coord_ls) / test_set.__len__()
        te_conf_loss = sum(loss_conf_ls) / test_set.__len__()
        te_cls_loss = sum(loss_cls_ls) / test_set.__len__()
        print("Epoch: {}/{}, Lr: {}, Test Loss:{:.2f} (Coord:{:.2f} Conf:{:.2f} Cls:{:.2f})".format(
            epoch + 1,
            num_epoches,
            optimizer.param_groups[0]['lr'],
            te_loss,
            te_coord_loss,
            te_conf_loss,
            te_cls_loss))
        if te_loss < best_loss:
            best_loss = te_loss
            best_model_wts = copy.deepcopy(model.state_dict())
'''
torch.save(best_model_wts, 'trained_models/yolov2_voc_weights1.pth')

Epoch: 26/50, Iteration: 1/156, Lr: 1e-05, Train Loss:3.16 (Coord:0.54 Conf:2.04 Cls:0.57)
Epoch: 26/50, Iteration: 11/156, Lr: 1e-05, Train Loss:2.20 (Coord:0.37 Conf:1.59 Cls:0.24)
Epoch: 26/50, Iteration: 21/156, Lr: 1e-05, Train Loss:2.48 (Coord:0.37 Conf:1.63 Cls:0.48)
Epoch: 26/50, Iteration: 31/156, Lr: 1e-05, Train Loss:3.40 (Coord:0.51 Conf:2.01 Cls:0.88)
Epoch: 26/50, Iteration: 41/156, Lr: 1e-05, Train Loss:3.48 (Coord:0.48 Conf:2.35 Cls:0.65)
Epoch: 26/50, Iteration: 51/156, Lr: 1e-05, Train Loss:3.30 (Coord:0.48 Conf:2.21 Cls:0.61)
Epoch: 26/50, Iteration: 61/156, Lr: 1e-05, Train Loss:2.20 (Coord:0.34 Conf:1.54 Cls:0.31)
Epoch: 26/50, Iteration: 71/156, Lr: 1e-05, Train Loss:2.51 (Coord:0.30 Conf:1.79 Cls:0.42)
Epoch: 26/50, Iteration: 81/156, Lr: 1e-05, Train Loss:2.35 (Coord:0.31 Conf:1.59 Cls:0.45)
Epoch: 26/50, Iteration: 91/156, Lr: 1e-05, Train Loss:2.84 (Coord:0.40 Conf:1.93 Cls:0.51)
Epoch: 26/50, Iteration: 101/156, Lr: 1e-05, Train Loss:4.03 (Coord:0.53 Conf:2.7

In [5]:
best_loss

tensor(1.1560, device='cuda:0', grad_fn=<AddBackward0>)